In [1]:
import pymongo

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['newsTemp']   # Enter your database name
mycol = mydb['articles']      # Enter your collection name

In [13]:
records = mycol.find()

In [4]:
tempHt = {}
uniqueWordCount = {}

In [15]:
for row in records:
    content = row['content']
    content = content.split()
    content = list(filter(lambda x: x.isalnum(), content))
    content = list(map(lambda x: x.lower(), content))
    
    wordCount = {}
    for word in content:
        temp = wordCount.pop(word, False)
        if temp is False:
            wordCount[word] = 1
        else:
            wordCount[word] = temp + 1
            
        temp = uniqueWordCount.pop(word, False)
        if temp is False:
            uniqueWordCount[word] = 1
        else:
            uniqueWordCount[word] = temp + 1
            
    temp = tempHt.pop(row['date'], False)
    if temp is False:
        tempHt[row['date']] = wordCount
    else:
        for word in wordCount.keys():
            temp1 = temp.pop(word, False)
            if temp1 is False:
                temp[word] = wordCount[word]
            else:
                temp[word] = temp1 + wordCount[word]
                
        tempHt[row['date']] = temp
        
        
    #print content
    #break

In [39]:
print uniqueWordCount['washington']

11662


In [33]:
res = [None] * len(uniqueWordCount.keys())

In [34]:
i = 0
for word in uniqueWordCount.keys():
    tempJSON = {'words': word}
    dateHt = {}
    for eachDate in tempHt.keys():
        wordsForDate = tempHt[eachDate]
        tempCount = wordsForDate.pop(word, False)
        if tempCount is not False:
            dateHt[eachDate] = tempCount
            wordsForDate[word] = tempCount
            
    tempJSON['countByDate'] = dateHt
    res[i] = tempJSON
    i += 1

In [37]:
print res[1200]

{'countByDate': {u'2017-04-06': 1, u'2017-04-14': 1, u'2017-05-03': 1, u'2015-01-26': 1, u'2016-08-15': 1, u'2017-03-12': 1, u'2017-05-13': 1, u'2017-03-13': 1, u'2017-03-23': 1, u'2016-06-03': 1, u'2016-08-08': 4, u'2016-02-23': 1, u'2017-03-09': 1, u'2017-02-05': 3, u'2015-06-22': 1, u'2017-02-10': 1, u'2017-02-09': 1, u'2015-04-14': 1, u'2017-01-13': 1}, 'words': u'handbags'}


In [38]:
newDB = myclient['counts']  # New databse
newCol = newDB['wordCount'] # New collection
x = newCol.insert_many(res)